In [3]:
import pandas as pd
import re
import spacy
from tqdm import tqdm

In [10]:
# Load data
df = pd.read_csv("../data/raw/linkedin_jobs.csv")

In [12]:
nlp = spacy.load("en_core_web_sm")
disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ['tokenizer', 'tagger', 'lemmatizer', 'attribute_ruler']]
for pipe in disabled_pipes:
    nlp.disable_pipe(pipe)

# Text Preprocessing
# 1. Text Normalization
df = df.map(lambda x: x.lower())
df = df.drop_duplicates()

# 2. Remove Stop Words and Punctuations
def preprocess_text(text):
    doc = nlp(text)
    return " ".join([word.lemma_ for word in doc if not word.is_stop and word.is_alpha])

texts = []
for i, row in tqdm(df.iterrows()):
    texts.append(preprocess_text(row['job_description'])+"\n")

with open("../data/interim/jobs.txt", "w") as f:
    f.writelines(texts)